In [56]:
:opt no-lint

# 28 Basic Libraries
## 28.1 Basic libraries and data structures
The chapter gives hints on making decisions about the optimal data structures for a programs.
- with benchmarks
- guidelines on when WHNF or NF are suitable for benchmarking
- define constant applicative forms and explain argument saturation.

## 28.2 Benchmarking with Criterion
A library for benchmarking is `criterion`, which offers two functions:
- `whnf`: evaluates to the first data constructor
- `nf`: fully evaluating the `a` as well as the first data constructor

Often the WHNF is enough for a benchmarch, but sometimes the computation to be measured is interleaved by constructors, hence the NF is required.
Both functions takes the computation as a function in order to prevent sharing.
## 28.3 Profiling your programs
GHC profiling is controlled by options:
- `-prof` : enables profiling
- `-fprof-auto`: auto-assigns bindings to a cost center

Moreover, it can measure the heap size.
## 28.4 Constant applicative forms
To talk about memory usage and sharing, we also have to talk about CAFs: constant applicative forms. CAFs are expressions that have no free variables and are held in memory to be shared with all other expressions in a module.
CAFs include:
- Values.
- Partially applied functions without named arguments.
- Fully applied functions.

Point-free, top-level declarations will be CAFs, but pointful ones will not.
## 28.5 Map
Maps offer fast key lookup (thanks to `Ord` constraint).
`HashMap` and `IntMap` are specialized for `Int` keys. `Vector` is appropriate for memory density and good locality use cases instead.
## 28.6 Set
Sets have a similar structure of maps, but the keys are the values.

In [57]:
import           Criterion.Main
import qualified Data.Map       as M
import qualified Data.Set       as S

bumpIt (i, v) = (i + 1, v + 1)

m :: M.Map Int Int
m = M.fromList $ take 10000 stream
  where
    stream = iterate bumpIt (0, 0)

s :: S.Set Int
s = S.fromList $ take 10000 stream
  where
    stream = iterate (+1) 0

lookupMap :: Int -> Maybe Int
lookupMap i = M.lookup i m

lookupSet :: Int -> Maybe Int
lookupSet i = S.lookupIndex i s

defaultMain
  [ bench "lookup map" $
    whnf lookupMap 9999
  , bench "lookup set" $
    whnf lookupSet 9999
  ]

: 

## 28.7 Sequence
#### Pros
- fast append both on the back and on the front
- fast tail access

#### Cons
- lower memory density than `Vector` (it's a persistent data structure)
- sometimes, concatentation is slower than `[]`

## 28.8 Vector
It is a sliced wrapper on top of `Array` data type, and it comes with many variants: boxed, unboxed, immutable, mutable, and storable vectors.

Use cases:
- memory efficienty
- access to elements via index
- uniform access performance
- write once, many reads

Multiple updates on a vector can be optimized with loop fusion which merges them all into a single pass.
Alternatively, the batch update operator (`//`) that allows to modify several elements of a vector at once.
Even faster updates can be achieved by the `ST` monad. It can be thought of as a mutable variant of the strict `State` monad, or as `IO` restricted exclusively to mutation, which is guaranteed safe. Thus, it manages to perform a mutation but still maintain referential transparency.
### Exercises: Vector

In [58]:
import           Criterion.Main
import qualified Data.Vector         as V
import qualified Data.Vector.Unboxed as U

v :: V.Vector Int
v = V.fromList [1..1000]

u :: U.Vector Int
u = U.fromList [1..1000]

defaultMain
  [ bench "slicing boxed" $
    whnf (V.head . V.slice 100 900) v
  , bench "slicing unboxed" $
    whnf (U.head . U.slice 100 900) u
  ]

: 

## 28.9 String types
### String
Alias for `[Char]`
### Text
- Compact representation in memory.
- Efficient indexing into the string.
- Encoded as UTF-16

### ByteString
It is a sequence of bytes backed by a vector of `Word8`. It’s easy to use via the `OverloadedStrings` extension.
Also useful for avoiding the overhead of encoding conversions. The conversion to `String` should be done with functions from `Data.ByteString.UTF8`.
## 28.10 Chapter exercises
### Difference list

In [59]:
newtype DList a = DL { unDL :: [a] -> [a] }

1. 

In [60]:
empty :: DList a
empty = DL id
{-# INLINE empty #-}

Pragmas of type INLINE
are not supported.

2. 

In [61]:
singleton :: a -> DList a
singleton = DL . (:)
{-# INLINE singleton #-}

Pragmas of type INLINE
are not supported.

3. 

In [62]:
toList :: DList a -> [a]
toList = ($[]) . unDL
{-# INLINE toList #-}

Pragmas of type INLINE
are not supported.

4. Prepend a single element to a dlist.

In [63]:
infixr `cons`
cons :: a -> DList a -> DList a
cons x xs = DL ((x:) . unDL xs)
{-# INLINE cons #-}

Pragmas of type INLINE
are not supported.

5. Append a single element to a dlist.

In [64]:
infixl `snoc`
snoc :: DList a -> a -> DList a
snoc xs x = DL (unDL xs . (x:))
{-# INLINE snoc #-}

Pragmas of type INLINE
are not supported.

6. Append dlists.

In [65]:
append :: DList a -> DList a -> DList a
append xs ys = DL (unDL xs . unDL ys)
{-# INLINE append #-}

Pragmas of type INLINE
are not supported.

Benchmarks:

In [66]:
schlemiel :: Int -> [Int]
schlemiel i = go i []
  where go 0 xs = xs
        go n xs = go (n-1) ([n] ++ xs)

consDlist :: Int -> [Int]
consDlist i = toList $ go i empty
  where go 0 xs = xs
        go n xs = go (n-1) (cons n xs)

snocDlist :: Int -> [Int]
snocDlist i = toList $ go i empty
  where go 0 xs = xs
        go n xs = go (n-1) (snoc xs n)

appendDlist :: Int -> [Int]
appendDlist i = toList $ go i empty
  where go 0 xs = xs
        go n xs = go (n-1) (singleton n `append` xs)

main :: IO ()
main = defaultMain
  [ bench "concat list" $ whnf schlemiel 1234567
  , bench "cons dlist" $ whnf consDlist 1234567
  , bench "snoc dlist" $ whnf snocDlist 1234567
  , bench "append dlist" $ whnf appendDlist 1234567
  ]
  
main

: 

### A simple queue

In [67]:
data Queue a = Queue
    { enqueue :: [a]
    , dequeue :: [a]
    } deriving (Eq, Show)

push :: a -> Queue a -> Queue a
push x q = case enqueue q of
  [] -> Queue [] [x]
  xs -> Queue [x] xs

pop :: Queue a -> Maybe (a, Queue a)
pop q = case dequeue q of
  []     -> Nothing
  (x:xs) -> Just (x, Queue [] xs)